In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [56]:
"""
卷积后，池化后尺寸计算公式：
((图像尺寸 - 卷积核尺寸 + 2 * 填充值) / 步长) + 1
((图像尺寸 - 池化窗尺寸 + 2 * 填充值) / 步长) + 1


感受野计算公式：
F(i) = (F(i + 1) - 1) * stride + kernel_size
"""


def conv2D_calculation(X, K, S=1, P=0, B=0):
    """
    X: input size
    K: kernel_size
    S: Step
    P: padding
    B: Bias
    """
    h, w = K.shape
    Y = np.zeros(
            (int(((X.shape[0] - h + 2 * P) / S ) + 1), 
            (int((X.shape[1] - w + 2 * P) / S ) + 1))
        )
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y + B



def Maxpool2D_calculation(X, K, S=1, P=0, B=0, mode='max'):
    """
    X: input size
    K: pool_size
    S: Step
    P: padding
    B: Bias
    """
    p_h, p_w = K
    Y = np.zeros(
            (int(((X.shape[0] - p_h + 2 * P) / S ) + 1), 
            (int((X.shape[1] - p_w + 2 * P) / S ) + 1))
        )
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y + B

In [57]:
X = np.array([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = np.array([[0.0, 1.0], [2.0, 3.0]])
print('no bias: \n', conv2D_calculation(X, K))
print('bias: \n', conv2D_calculation(X, K, B=2))

no bias: 
 [[19. 25.]
 [37. 43.]]
bias: 
 [[21. 27.]
 [39. 45.]]


In [58]:
X = torch.tensor([[0.0, 1.0, 2.0, 3.0], [3.0, 4.0, 5.0, 6.0], [6.0, 7.0, 8.0, 9.0],
                 [9.0, 10.0, 11.0, 12.0]])
Maxpool2D_calculation(X, (2, 2))

array([[ 4.,  5.,  6.],
       [ 7.,  8.,  9.],
       [10., 11., 12.]])

In [59]:
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 3.,  4.,  5.,  6.],
        [ 6.,  7.,  8.,  9.],
        [ 9., 10., 11., 12.]])

In [60]:
Maxpool2D_calculation(X, (2, 2), mode='avg')

array([[ 2.,  3.,  4.],
       [ 5.,  6.,  7.],
       [ 8.,  9., 10.]])

In [62]:
def drop_out(X, prob):
    """
    X: input size
    prob: drop_rate
    """
    assert 0 <= prob <= 1
    if prob == 1:
        return torch.zeros_like(X)
    if prob == 0:
        return X
    mask = (torch.randn(X.shape) > prob).float()
    return mask * X / (1.0 - prob)

In [66]:
X = torch.arange(1, 17, dtype=torch.float32).reshape(2, 8)
print(X)
print(drop_out(X, 0))
print(drop_out(X, 0.5))
print(drop_out(X, 1))

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12., 13., 14., 15., 16.]])
tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12., 13., 14., 15., 16.]])
tensor([[ 0.,  0.,  6.,  0.,  0.,  0.,  0., 16.],
        [ 0., 20., 22.,  0., 26., 28.,  0.,  0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


## ***AlexNet***

In [3]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.BatchNorm2d(num_features=96), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=1), nn.BatchNorm2d(num_features=256), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(), 
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten()
)

X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output: \n', X.shape)

Conv2d output: 
 torch.Size([1, 96, 54, 54])
BatchNorm2d output: 
 torch.Size([1, 96, 54, 54])
ReLU output: 
 torch.Size([1, 96, 54, 54])
MaxPool2d output: 
 torch.Size([1, 96, 26, 26])
Conv2d output: 
 torch.Size([1, 256, 24, 24])
BatchNorm2d output: 
 torch.Size([1, 256, 24, 24])
ReLU output: 
 torch.Size([1, 256, 24, 24])
MaxPool2d output: 
 torch.Size([1, 256, 11, 11])
Conv2d output: 
 torch.Size([1, 384, 11, 11])
ReLU output: 
 torch.Size([1, 384, 11, 11])
Conv2d output: 
 torch.Size([1, 384, 11, 11])
ReLU output: 
 torch.Size([1, 384, 11, 11])
Conv2d output: 
 torch.Size([1, 256, 11, 11])
ReLU output: 
 torch.Size([1, 256, 11, 11])
MaxPool2d output: 
 torch.Size([1, 256, 5, 5])
Flatten output: 
 torch.Size([1, 6400])


In [4]:
"""
当ceil_mode设置为True，即：如果尺寸不够，则自动填充向上取整
"""

net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.BatchNorm2d(num_features=96), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
    nn.Conv2d(96, 256, kernel_size=5, padding=1), nn.BatchNorm2d(num_features=256), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(), 
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
    nn.Flatten()
)

X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output: \n', X.shape)

Conv2d output: 
 torch.Size([1, 96, 54, 54])
BatchNorm2d output: 
 torch.Size([1, 96, 54, 54])
ReLU output: 
 torch.Size([1, 96, 54, 54])
MaxPool2d output: 
 torch.Size([1, 96, 27, 27])
Conv2d output: 
 torch.Size([1, 256, 25, 25])
BatchNorm2d output: 
 torch.Size([1, 256, 25, 25])
ReLU output: 
 torch.Size([1, 256, 25, 25])
MaxPool2d output: 
 torch.Size([1, 256, 12, 12])
Conv2d output: 
 torch.Size([1, 384, 12, 12])
ReLU output: 
 torch.Size([1, 384, 12, 12])
Conv2d output: 
 torch.Size([1, 384, 12, 12])
ReLU output: 
 torch.Size([1, 384, 12, 12])
Conv2d output: 
 torch.Size([1, 256, 12, 12])
ReLU output: 
 torch.Size([1, 256, 12, 12])
MaxPool2d output: 
 torch.Size([1, 256, 6, 6])
Flatten output: 
 torch.Size([1, 9216])
